In [1]:
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import backend as K
from keras.optimizers import SGD
from keras.callbacks import EarlyStopping



# dimensions of our images.
img_width, img_height = 224, 224

train_data_dir = 'data/data/train'
validation_data_dir = 'data/data/validation'
nb_train_samples = 1600
nb_validation_samples = 692
epochs = 10
batch_size = 16

if K.image_data_format() == 'channels_first':
    input_shape = (3, img_height, img_width)
else:
    input_shape = (img_height, img_width, 3)
    
# this is the augmentation configuration we will use for training
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1. / 255)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary')


model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=input_shape, name='conv_1'))
#model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3), name='conv_2'))
#model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3), name='conv_3'))
#model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(64, name='dense_1s'))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1, name='dense_2s'))
model.add(Activation('sigmoid'))

Using TensorFlow backend.


Found 1600 images belonging to 2 classes.
Found 695 images belonging to 2 classes.


In [2]:
#model.load_weights('weights_s.18-0.34.hdf5', by_name=True)
model.load_weights('first_try_adam.h5')

x = 1
if x:
    optimizer='adam'
else:
    optimizer=SGD(lr=0.0001, momentum=0.9, nesterov=True)

model.compile(loss='binary_crossentropy',
              optimizer=optimizer,
              metrics=['accuracy'])


epochs = 1

import keras
early_stopping = EarlyStopping(monitor='val_loss', patience=3, verbose=2, mode='auto')
save_model = keras.callbacks.ModelCheckpoint('weights_s.{epoch:02d}-{val_loss:.2f}.hdf5', save_best_only=True)
TensorBoard = keras.callbacks.TensorBoard(log_dir='./logs_s', batch_size=batch_size)

history = model.fit_generator(
    train_generator,
    steps_per_epoch=nb_train_samples // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=nb_validation_samples // batch_size,
    verbose = 1,
    callbacks=[save_model, TensorBoard])

model.save_weights('first_try_adam_dd.h5','w')

Epoch 1/1
100/100 [==============================] - 238s - loss: 0.2746 - acc: 0.8806 - val_loss: 0.2789 - val_acc: 0.9055


In [9]:
model.save_weights('first_try_adam.h5','w')

In [7]:
model.load_weights('first_try_adam.h5')


test_data_dir = 'data/test'
test_generator = test_datagen.flow_from_directory(
    test_data_dir,
    target_size=(img_height, img_width),
    batch_size=1,
    class_mode="binary",
    shuffle = False)

preds = model.predict_generator(test_generator, 1531,verbose = 1)
preds_rounded = []

for pred in preds:
    if (pred > .5):
        preds_rounded.append("1")
    else:
        preds_rounded.append("0")
        
preds_filenames = [int(x.replace("New Folder With Items/", "").replace(".jpg", "")) for x in test_generator.filenames]   

data = (list(zip(preds_filenames, preds_rounded)))

import pandas as pd
df_result = pd.DataFrame(data, columns=["name", "invasive"])
df_result = df_result.sort_values("name")
df_result.index = df_result["name"]
df_result = df_result.drop(["name"], axis=1)

df_result.to_csv("submission_01.csv", encoding="utf8", index=True)


Found 1531 images belonging to 1 classes.
1531/1531 [==============================] - 104s   


In [9]:
preds

array([[ 0.48470336],
       [ 0.50816661],
       [ 0.50916028],
       ..., 
       [ 0.53109843],
       [ 0.52650106],
       [ 0.5087626 ]], dtype=float32)

In [11]:
import cv2

ModuleNotFoundError: No module named 'cv2'